In [1]:
import numpy as np
import pandas as pd
#import tensorflow as tf


import contractions
import os
from collections import Counter
from bs4 import BeautifulSoup
# import torch
# from torch.utils.data import DataLoader, Dataset

# from transformers import BertForSequenceClassification, BertTokenizer
# import torch

# from torch.utils.data.sampler import SubsetRandomSampler
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset
# cuda = torch.device('cuda')


import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nlpaug
import nlpaug.augmenter.word as nlpaw

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
!pip install yfinance
import yfinance as yf

from pandas.tseries.offsets import DateOffset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harshiljhaveri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '../data/Transcripts'
df = pd.DataFrame()
companies = {'AMZN':'Amazon',
 'AAPL':'Apple',
 'INTC':'Intel',
 'MU':'Micron',
 'GOOGL': 'Alphabet',
 'MSFT': 'Microsoft',
 'AMD':'AMD',
 'ASML':'ASML',
 'NVDA':'NVIDIA',
 'CSCO':'Cisco'}

In [3]:
df = []
tf = 0
totalsummary = ''
totalqna = ''
for comp in os.listdir(path):
    if comp!='.DS_Store':
        company = comp
        files = os.listdir(path+'/'+company)
        for j in files:
            allinfo = []
            with open(path+'/'+company+'/'+j) as f:
                lines = f.readlines()
                summary = []
                qna = []
                i = 0
                while i<len(lines):
                    if lines[i]=='\n':
                        i+=1
                    elif lines[i][:3]=='===':
                        i+=1
                    elif 'operator' in lines[i].lower():
                        i+=3
                    elif lines[i][:3] == '---':
                        i+=1
                    elif 'presentation' in lines[i].lower():
                        i+=2
#                         print(j)
                        while i<len(lines):
#                             print(lines[i])
                            if '===' in lines[i]:
                                break
                            if 'Questions and Answers' in lines[i]:
                                break
                            elif lines[i]=='\n':
                                i+=1
                                continue
                            elif 'operator' in lines[i].lower():
                                i+=3
                            elif '---' in lines[i]:
                                i+=1
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() in lines[i].lower():
                        i+=2
                        while lines[i][:3]!='---':
                            if lines[i]=='\n':
                                i+=1
                                continue
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif 'Questions and Answers' in lines[i]:
                        i+=1
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=2
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() not in lines[i].lower():
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=3
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    else:
                        i+=1
                
                if lines[0][0] == 'Q':
                    quarter = lines[0].split(' ')[0]
                    year = lines[0].split(' ')[1]
                else:
                    quarter = lines[1].split(' ')[0]
                    year = lines[1].split(' ')[1]
                cleansummary = []
#                 print(j)
                for i in range(len(summary)):
                    if '--' in summary[i]:
                        i+=1
                    elif 'Operator' in summary[i]:
                        i+=3
                    elif '[' in summary[i]:
#                         print(i, summary[i])
                        i+=1
                    else:
                        cleansummary.append(summary[i])
                summary_para = ' '.join(cleansummary).rstrip()
                qna_para = ' '.join(qna).rstrip()
                totalsummary += summary_para
                totalqna = qna_para
                allinfo.append(quarter)
                allinfo.append(year)
                allinfo.append(companies[company])
                allinfo.append(summary_para)
                allinfo.append(qna_para)
                df.append(allinfo)

In [4]:
earnings_transcript_df = pd.DataFrame(df)
earnings_transcript_df.columns =['Quarter','Year','Company','Summary','Q&A']

In [5]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q4,2019,Amazon,"Hello, and welcome to our Q4 2019 f...",Great. Really appreciate this. Just ...
1,Q1,2016,Amazon,"Good day, everyone, and welcome to t...","Thanks a lot. Lots here, but interna..."
2,Q3,2019,Amazon,"Hello, and welcome to our Q3 2019 F...",I wanted to know if I can go a littl...
3,Q1,2020,Amazon,"For opening remarks, I will be turning the cal...","Great. First, I just wanted to ask, within the..."
4,Q1,2019,Amazon,"Hello, and welcome to our Q1 2019 f...",Okay. I'm going to ask 2 questions b...
...,...,...,...,...,...
183,Q1,2018,Cisco,Welcome to Cisco's First Quarter of ...,Our first question comes from the li...
184,Q3,2018,Cisco,Welcome to Cisco's Third Quarter Fis...,James Faucette from Morgan Stanley I...
185,Q3,2017,Cisco,Welcome to Cisco Systems' Third Quar...,I wanted to see if we could understa...
186,Q3,2020,Cisco,"Thanks, Michelle. Welcome, everyone, to Cisco...",Our first question comes from Paul Silverstein...


In [6]:
words = Counter(totalsummary.split(' '))
words += Counter(totalqna.split(' '))

In [7]:
def getAvgTxtLen(df):
    avg_length_summary_after_preprocessing = str(df['Summary'].str.len().mean()) 
    avg_length_qna_after_preprocessing = str(df['Q&A'].str.len().mean()) 
    print(avg_length_summary_after_preprocessing)
    print(avg_length_qna_after_preprocessing)

In [8]:
def strip_html_tags(text): 
    soup = BeautifulSoup(text, "html.parser") 
    stripped_text = soup.get_text(separator=" ") 
    return stripped_text 
    
# Make lowercase
earnings_transcript_df["Summary"] = earnings_transcript_df["Summary"].str.lower()
earnings_transcript_df["Q&A"] = earnings_transcript_df["Q&A"].str.lower()

# Remove URLS
earnings_transcript_df['Summary']= earnings_transcript_df['Summary'].str.replace('http(s)?:\/\/.*', '', regex=True)
earnings_transcript_df['Q&A']= earnings_transcript_df['Q&A'].str.replace('http(s)?:\/\/.*', '', regex=True)
print("Remove URLS")
getAvgTxtLen(earnings_transcript_df)

# remove HTML tags
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: strip_html_tags(x))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: strip_html_tags(x))
print("remove HTML tags")
getAvgTxtLen(earnings_transcript_df)

# remove extra whitespaces
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace(r'/\s/\s+/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace(r'/\s/\s+/g', ' ', regex=True)
print("remove extra whitespaces")
getAvgTxtLen(earnings_transcript_df)

# remove contractions
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
print("remove contractions")
getAvgTxtLen(earnings_transcript_df)

# Remove characters following digits such as 1st, 2nd or 3rd
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('\b\d+\w+\b', '', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('\b\d+\w+\b', '', regex=True)
print("Remove characters following digits such as 1st, 2nd or 3rd")
getAvgTxtLen(earnings_transcript_df)

# Replace hyphens with space
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace('/\b-\b/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace('/\b-\b/g', ' ', regex=True)
print("Replace hyphens with space")
getAvgTxtLen(earnings_transcript_df)

# remove non alphabetical characters
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('[^a-zA-Z0-9 ]', ' ')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('[^a-zA-Z0-9 ]', ' ')
print("remove non alphabetical characters")
getAvgTxtLen(earnings_transcript_df)


Remove URLS
16292.622340425532
28507.35106382979
remove HTML tags
16292.617021276596
28088.063829787236
remove extra whitespaces
16292.617021276596
28088.063829787236
remove contractions
16065.97340425532
27840.127659574468
Remove characters following digits such as 1st, 2nd or 3rd
16065.97340425532
27840.127659574468
Replace hyphens with space
16065.97340425532
27840.127659574468
remove non alphabetical characters
16065.97340425532
27840.127659574468


In [9]:
earnings_transcript_df.iloc[0,2]

'Amazon'

In [10]:
stop_words = stopwords.words('english')
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')

earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')

print("Stopwords")
getAvgTxtLen(earnings_transcript_df)
# Lemmatize the Dataset
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_review(text):
    return ' '.join(lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text))

earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lemmatize_review)
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lemmatize_review)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')
print("Lemmatize")
getAvgTxtLen(earnings_transcript_df)



Stopwords
11871.898936170213
16657.393617021276
Lemmatize
11406.505319148937
15969.808510638299


In [11]:
avg_length_summary_after_preprocessing = str(earnings_transcript_df['Summary'].str.len().mean()) 
avg_length_qna_after_preprocessing = str(earnings_transcript_df['Q&A'].str.len().mean()) 
print(avg_length_summary_after_preprocessing)
print(avg_length_qna_after_preprocessing)

11406.505319148937
15969.808510638299


In [12]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...
1,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...
2,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...
3,Q1,2020,Amazon,open remark turn call director investor relati...,great first want ask within 4 billion covid re...
4,Q1,2019,Amazon,hello welcome q1 2019 financial result confere...,okay go ask 2 question first one really simple...
...,...,...,...,...,...
183,Q1,2018,Cisco,welcome cisco first quarter fiscal year 2018 f...,first question come line pierre ferragu sanfor...
184,Q3,2018,Cisco,welcome cisco third quarter fiscal year 2018 f...,jam faucette morgan stanley investment researc...
185,Q3,2017,Cisco,welcome cisco systems third quarter fiscal yea...,want see could understand guidance kelly chuck...
186,Q3,2020,Cisco,thank michelle welcome everyone cisco third qu...,first question come paul silverstein cowen com...


In [13]:
# Running augmentations
augdf = earnings_transcript_df.copy()
aug = nlpaw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=10, aug_max=60, aug_p=0.3)
for i in range(len(augdf)):
    k = aug.augment(earnings_transcript_df.iloc[i]['Summary'],n=1)
    augdf['Summary'][i] = k[0]
    k = aug.augment(earnings_transcript_df.iloc[i]['Q&A'],n=1)
    augdf['Q&A'][i] = k[0]

earnings_transcript_df = earnings_transcript_df.append(augdf)
earnings_transcript_df = earnings_transcript_df.reset_index().drop(columns = ['index'])

In [14]:
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...
1,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...
2,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...
3,Q1,2020,Amazon,open remark turn call director investor relati...,great first want ask within 4 billion covid re...
4,Q1,2019,Amazon,hello welcome q1 2019 financial result confere...,okay go ask 2 question first one really simple...
...,...,...,...,...,...
371,Q1,2018,Cisco,welcome cisco first quarter fiscal year 2018 f...,first question come line pierre ferragu sanfor...
372,Q3,2018,Cisco,welcome cisco third tail fiscal year 2018 fina...,jam faucette morgan stanley investment researc...
373,Q3,2017,Cisco,welcome cisco systems third quarter financial ...,want see could understand guidance kelly chuck...
374,Q3,2020,Cisco,thank michelle welcome everyone cisco third qu...,first question come saul silverstein cowen com...


In [15]:
today = pd.to_datetime("today")

# Consolidated Dataframe with sentiments for all companies
finance_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])

company_tickers = ['NVDA','MU','INTC','GOOGL','CSCO','ASML','AMZN','AMD','AAPL','MSFT']

price_change_perc_threshold_for_sentiment = 3.0

for ticker in company_tickers:
    
    # Get List of dates for Earnings call
    ticker_data = yf.Ticker(ticker)
    ticker_data.earnings_dates.reset_index(inplace=True)
    
    #Format Date column
    ticker_data.earnings_dates["Earnings Date"] = ticker_data.earnings_dates["Earnings Date"].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # Filter dates within window of our Transcripts
    earnings_dates_df = ticker_data.earnings_dates[(ticker_data.earnings_dates["Earnings Date"]<=today.strftime('%Y-%m-%d')) & (ticker_data.earnings_dates["Earnings Date"]>="2015-12-01")]
    earning_call_dates= earnings_dates_df.iloc[:,0]

    #print(earning_call_dates)
    
    prepared_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])
    
    historical_data = yf.download(ticker, start="2015-12-01", end="2022-11-21", interval="1d")

    historical_data.reset_index(inplace=True)

    for date in earning_call_dates:
        # Compute Quarters for each date to join finance df with Transcripts DF created in above cells
        month=pd.to_datetime(date).month
        year=pd.to_datetime(date).year
        if month<=3:
            quarter="Q1"
        elif month<=6:
            quarter="Q2"
        elif month<=9:
            quarter="Q3"
        else:
            quarter="Q4"
                
        # Compute % change by looking at T-2 and T+2 window from earnings call date
        # i.e. if Earnings call is 27 Oct 2022 => compute:
        # => (Closing_price['29 Oct'] - Closing_price['27 Oct']) * 100 / (Closing_price['27 Oct'])
        
        index= historical_data.index[historical_data["Date"]==pd.to_datetime(date)]
        tplus2= float(historical_data.loc[index+2,"Adj Close"])
        tminus2 = float(historical_data.loc[index-2,"Adj Close"])
        change = (tplus2 - tminus2)*100/tminus2
        
        # Computer sentiment based on price change percentage threshold
        if abs(change)< price_change_perc_threshold_for_sentiment:
            label="Neutral"
        elif change> price_change_perc_threshold_for_sentiment:
            label="Positive"
        else:
            label="Negative"
        prepared_df = prepared_df.append({"Company":companies[ticker],"date":date,"Quarter":quarter, "Year": year, "percent change (between t-2 and t+2)": change, "Sentiment": label}, ignore_index=True)
    prepared_df.reset_index(drop=True)
    prepared_df.to_csv("..\\data\\OHLC\\"+ticker+".csv", encoding='utf-8')
    finance_df=pd.concat([finance_df, prepared_df], ignore_index=True)
finance_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Company,date,Quarter,Year,percent change (between t-2 and t+2),Sentiment
0,NVIDIA,2022-11-16,Q4,2022,-5.437256,Negative
1,NVIDIA,2022-08-24,Q3,2022,-4.543851,Negative
2,NVIDIA,2022-05-25,Q2,2022,11.320858,Positive
3,NVIDIA,2022-02-16,Q1,2022,-2.575518,Neutral
4,NVIDIA,2021-11-17,Q4,2021,9.858471,Positive
...,...,...,...,...,...,...
275,Microsoft,2017-01-26,Q1,2017,2.534617,Neutral
276,Microsoft,2016-10-20,Q4,2016,5.792598,Positive
277,Microsoft,2016-07-19,Q3,2016,3.910607,Positive
278,Microsoft,2016-04-21,Q2,2016,-7.589991,Negative


In [16]:
finance_df.Year = finance_df.Year.astype('int')
earnings_transcript_df.Year = earnings_transcript_df.Year.astype('int')

In [17]:
df_merged = pd.merge(earnings_transcript_df, finance_df, on=['Company', 'Year', 'Quarter'], how='inner')

In [18]:
df_merged

,Quarter,Year,Company,Summary,Q&A,date,percent change (between t-2 and t+2),Sentiment
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,2019-10-24,0.642791,Neutral
1,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,2019-10-24,0.642791,Neutral
2,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...,2016-01-28,-4.397507,Negative
3,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot destiny international retail revenue...,2016-01-28,-4.397507,Negative
4,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...,2019-07-25,-4.113337,Negative
...,...,...,...,...,...,...,...,...
361,Q3,2017,Cisco,welcome cisco systems third quarter financial ...,want see could understand guidance kelly chuck...,2017-08-16,-4.616820,Negative
362,Q3,2020,Cisco,thank michelle welcome everyone cisco third qu...,first question come paul silverstein cowen com...,2020-08-12,-10.957449,Negative
363,Q3,2020,Cisco,thank michelle welcome everyone cisco third qu...,first question come saul silverstein cowen com...,2020-08-12,-10.957449,Negative
364,Q4,2016,Cisco,thank sam welcome everyone cisco fourth quarte...,thank first question come line vijay bhagavath...,2016-11-16,-3.793446,Negative
